In [1]:
!pip install transformers pandas numpy torch nltk

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report, accuracy_score
import nltk

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

df = pd.read_csv('youtube_comments.csv')
comments = df['komentar'].values

# Mengatasi nilai NaN dengan mengisi dengan string kosong
comments = [str(comment) if not pd.isna(comment) else '' for comment in comments]

# Fungsi untuk preprocessing teks
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    ps = PorterStemmer()
    tokens = [ps.stem(word) for word in tokens]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

processed_comments = [preprocess_text(comment) for comment in comments]

# Inisialisasi VADER
sia = SentimentIntensityAnalyzer()

# Fungsi untuk memberikan label sentimen
def label_sentiment_vader(comment):
    score = sia.polarity_scores(comment)
    if score['compound'] >= 0.05:
        return 'positif'
    elif score['compound'] <= -0.05:
        return 'negatif'
    else:
        return 'netral'

labeled_comments = [label_sentiment_vader(comment) for comment in processed_comments]

df = pd.DataFrame({'comment': processed_comments, 'label': labeled_comments})
df.to_csv('youtube_comments_labeled.csv', index=False)

comments = df['comment'].values
labels = df['label'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

train_sentences, test_sentences, train_labels, test_labels = train_test_split(comments, encoded_labels, stratify = encoded_labels)

# Hyperparameters of the model
vocab_size = 5000 # increased vocabulary size
embedding_dim = 128
max_length = 200
padding_type='post'
trunc_type='post'
oov_tok = '<OOV>'

# Tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# Convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# Convert test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

# Model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
    keras.layers.GlobalMaxPool1D(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(3, activation='softmax')
])

# Compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              metrics=['accuracy'])

# Model summary
model.summary()

num_epochs = 10
history = model.fit(train_padded, train_labels,
                    epochs=num_epochs, verbose=1,
                    validation_split=0.1,
                    batch_size=64)

# Evaluation
prediction = model.predict(test_padded)
pred_labels = np.argmax(prediction, axis=1)

print("Accuracy of prediction on test set : ", accuracy_score(test_labels, pred_labels))
print(classification_report(test_labels, pred_labels, target_names=encoder.classes_))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 bidirectional (Bidirection  (None, 200, 256)          263168    
 al)                                                             
                                                                 
 global_max_pooling1d (Glob  (None, 256)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 3

In [14]:
# Predict on new sentences
sentence = ["This community is filled with kind and caring people.",
            "They have not got a dictionary.",
            "I found it difficult to follow this material"]

sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, padding='post', maxlen=max_length)
prediction = model.predict(padded)
pred_labels = np.argmax(prediction, axis=1)

for i in range(len(sentence)):
    print(sentence[i])
    sentiment = encoder.inverse_transform([pred_labels[i]])[0]
    print("Predicted sentiment : ", sentiment)

1/1 [==============================] - 0s 188ms/step
This community is filled with kind and caring people.
Predicted sentiment :  positif
They have not got a dictionary.
Predicted sentiment :  netral
I found it difficult to follow this material
Predicted sentiment :  negatif


In [16]:
!pip freeze > requirements1.txt